In [146]:
import numpy as np
import openai
import pandas as pd
import unicodedata

In [9]:
# load CSV
df = pd.read_csv('/Users/ohmpatel/Downloads/nytcrosswords.csv', encoding="cp1252")

In [90]:
prompts = df['Clue'].iloc[:25]

In [134]:
def find_answer(sentence, length):
    for word in sentence.split():
        if len(word) >= 3 and word.upper() == word:
            return word
    return "NULL"

In [144]:
def preprocess_clue_fn(clue):
    clue = str(clue)

    # https://stackoverflow.com/questions/517923/what-is-the-best-way-to-remove-accents-normalize-in-a-python-unicode-string
    clue = ''.join(c for c in unicodedata.normalize('NFD', clue) if unicodedata.category(c) != 'Mn')

    clue = re.sub("\x17|\x18|\x93|\x94|“|”|''|\"\"", "\"", clue)
    clue = re.sub("\x85|…", "...", clue)
    clue = re.sub("\x91|\x92|‘|’", "'", clue)

    clue = re.sub("‚", ",", clue)
    clue = re.sub("—|–", "-", clue)
    clue = re.sub("¢", " cents", clue)
    clue = re.sub("¿|¡|^;|\{|\}", "", clue)
    clue = re.sub("÷", "division", clue)
    clue = re.sub("°", " degrees", clue)

    euro = re.search("^£[0-9]+(,*[0-9]*){0,}| £[0-9]+(,*[0-9]*){0,}", clue)
    if euro:
        num = clue[:euro.end()]
        rest_clue = clue[euro.end():]
        clue = num + " Euros" + rest_clue
        clue = re.sub(", Euros", " Euros", clue)
        clue = re.sub("Euros [Mm]illion", "million Euros", clue)
        clue = re.sub("Euros [Bb]illion", "billion Euros", clue)
        clue = re.sub("Euros[Kk]", "K Euros", clue)
        clue = re.sub(" K Euros", "K Euros", clue)
        clue = re.sub("£", "", clue)

    clue = re.sub(" *\(\d{1,},*\)$| *\(\d{1,},* \d{1,}\)$", "", clue)

    clue = re.sub("&amp;", "&", clue)
    clue = re.sub("&lt;", "<", clue)
    clue = re.sub("&gt;", ">", clue)

    clue = re.sub("e\.g\.|for ex\.", "for example", clue)
    clue = re.sub(": [Aa]bbreviat\.|: [Aa]bbrev\.|: [Aa]bbrv\.|: [Aa]bbrv|: [Aa]bbr\.|: [Aa]bbr", " abbreviation", clue)
    clue = re.sub("abbr\.|abbrv\.", "abbreviation", clue)
    clue = re.sub("Abbr\.|Abbrv\.", "Abbreviation", clue)
    clue = re.sub("\(anag\.\)|\(anag\)", "(anagram)", clue)
    clue = re.sub("org\.", "organization", clue)
    clue = re.sub("Org\.", "Organization", clue)
    clue = re.sub("Grp\.|Gp\.", "Group", clue)
    clue = re.sub("grp\.|gp\.", "group", clue)
    clue = re.sub(": Sp\.", " (Spanish)", clue)
    clue = re.sub("\(Sp\.\)|Sp\.", "(Spanish)", clue)
    clue = re.sub("Ave\.", "Avenue", clue)
    clue = re.sub("Sch\.", "School", clue)
    clue = re.sub("sch\.", "school", clue)
    clue = re.sub("Agcy\.", "Agency", clue)
    clue = re.sub("agcy\.", "agency", clue)
    clue = re.sub("Co\.", "Company", clue)
    clue = re.sub("co\.", "company", clue)
    clue = re.sub("No\.", "Number", clue)
    clue = re.sub("Mo\.", "Month", clue)
    clue = re.sub("mo\.", "month", clue)
    clue = re.sub("no\.", "number", clue)
    clue = re.sub(": [Vv]ar\.", " variable", clue)
    clue = re.sub("Subj\.", "Subject", clue)
    clue = re.sub("subj\.", "subject", clue)
    clue = re.sub("Subjs\.", "Subjects", clue)
    clue = re.sub("subjs\.", "subjects", clue)

    theme_clue = re.search("^.+\|[A-Z]{1,}", clue)
    if theme_clue:
        clue = re.sub("\|", " | ", clue)

    if "Partner of" in clue:
        clue = re.sub("Partner of", "", clue)
        clue = clue + " and ___"

    link = re.search("^.+-.+ [Ll]ink$", clue)
    if link:
        no_link = re.search("^.+-.+ ", clue)
        x_y = clue[no_link.start():no_link.end() - 1]
        x_y_lst = x_y.split("-")
        clue = x_y_lst[0] + " ___ " + x_y_lst[1]

    follower = re.search("^.+ [Ff]ollower$", clue)
    if follower:
        no_follower = re.search("^.+ ", clue)
        x = clue[:no_follower.end() - 1]
        clue = x + " ___"

    preceder = re.search("^.+ [Pp]receder$", clue)
    if preceder:
        no_preceder = re.search("^.+ ", clue)
        x = clue[:no_preceder.end() - 1]
        clue = "___ " + x

    if re.search("--[^A-Za-z]|--$", clue):
        clue = re.sub("--", "__", clue)
    if not re.search("_-[A-Za-z]|_-$", clue):
        clue = re.sub("_-", "__", clue)

    clue = re.sub("_{2,}", "___", clue)

    clue = re.sub("\?$", " (wordplay)", clue)

    nonverbal = re.search("\[[^0-9]+,* *[^0-9]*\]", clue)
    if nonverbal:
        clue = re.sub("\[|\]", "", clue)
        clue = clue + " (nonverbal)"

    if clue[:4] == "\"\"\" " and clue[-4:] == " \"\"\"":
        clue = "\"" + clue[4:-4] + "\""
    if clue[:4] == "''' " and clue[-4:] == " '''":
        clue = "'" + clue[4:-4] + "'"
    if clue[:3] == "\"\"\"" and clue[-3:] == "\"\"\"":
        clue = "\"" + clue[3:-3] + "\""
    if clue[:3] == "'''" and clue[-3:] == "'''":
        clue = "'" + clue[3:-3] + "'"

    return clue

<>:14: SyntaxWarning: invalid escape sequence '\{'
<>:30: SyntaxWarning: invalid escape sequence '\('
<>:36: SyntaxWarning: invalid escape sequence '\.'
<>:37: SyntaxWarning: invalid escape sequence '\.'
<>:38: SyntaxWarning: invalid escape sequence '\.'
<>:39: SyntaxWarning: invalid escape sequence '\.'
<>:40: SyntaxWarning: invalid escape sequence '\('
<>:41: SyntaxWarning: invalid escape sequence '\.'
<>:42: SyntaxWarning: invalid escape sequence '\.'
<>:43: SyntaxWarning: invalid escape sequence '\.'
<>:44: SyntaxWarning: invalid escape sequence '\.'
<>:45: SyntaxWarning: invalid escape sequence '\.'
<>:46: SyntaxWarning: invalid escape sequence '\('
<>:47: SyntaxWarning: invalid escape sequence '\.'
<>:48: SyntaxWarning: invalid escape sequence '\.'
<>:49: SyntaxWarning: invalid escape sequence '\.'
<>:50: SyntaxWarning: invalid escape sequence '\.'
<>:51: SyntaxWarning: invalid escape sequence '\.'
<>:52: SyntaxWarning: invalid escape sequence '\.'
<>:53: SyntaxWarning: invalid e

In [151]:
answers = []

client = Together(api_key='cab3e5ed2c07e6ca3f6c70af0928edac2e7f0d4bf138e392743fcd73f4b5e88f')

for i, prompt in enumerate(df['Clue']):
    clue = preprocess_clue_fn(df['Clue'][i])
    length = len(df['Word'][i])
    
    response = client.chat.completions.create(
        model="meta-llama/Llama-3-8b-chat-hf",
        messages=[{"role": "user", "content": f"The crossword clue is {clue}. The length of the answer is {length} characters. Write the answer in all caps and with no spaces."}]
    )
    answer = response.choices[0].message.content
    stripped = answer.translate(str.maketrans('', '', string.punctuation))
    answers.append(stripped)
    print(prompt, stripped)

Action done while saying "Good dog" The answer is PET
Mischief-makers The answer to the crossword clue Mischiefmakers with a length of 7 characters is TRICKERS
It might click for a writer I think I can help you with that

The answer to the crossword clue It might click for a writer with a length of 3 characters is PEN
Fall mo. SEP
Kind to Mother Nature The answer is ECO
Harris in the Country Music Hall of Fame The answer is CALTHOMPSON
Living ___ The answer is LARGE
Member of the superfamily Hominoidea The answer is APE
Haphazard The answer to the crossword clue Haphazard with a length of 9 characters is

ADHOCERLY
Some crumbly blocks The answer is COOK
Inits. for a theatrical hit I think I can help you with that

The answer to the crossword clue Inits for a theatrical hit with a length of 3 characters is likely ACT
Send away, in a way The answer is EXPELLED
Accomplished the task The answer is DONEIT
What wiggly lines in comics may represent The answer is BONDS
Cause of boom and bust? 

Guff The answer is NOD
Distributor of CARES Act funds The answer is IRS
___ Moines The answer is DES
Words on some flashcards, informally The answer is TERMS
Stuff The answer is GOODS
"___ are ..." I think I can help you with that

The answer to the crossword clue  are  with a length of 4 characters is ISEE
Academia, it's said The answer to the crossword clue Academia with a length of 10 characters is

UNIVERSITIES
TV host who was the subject of the documentary "You Laugh but It's True" The answer is RICH
Pot price I think I can help you with that

The answer to the crossword clue Pot price with a length of 4 characters is likely OZES
Its larva is eaten as a delicacy in the Mexican dish escamoles The answer is ANT
Suffix with carboxyl The answer is IDE
Loved, on social media The answer is LIKEDIT
"Olympus ___ Fallen" (2013 film) The answer is HAS
Quiet (down) The answer is HUSH
Defeated, in a way The answer is BEATEN
What Real Madrid and F.C. Barcelona play in The answer is FUTBOL
Cler

Possible result of a TMZ story I think I can help you with that

The possible answer to the crossword clue Possible result of a TMZ story with a length of 5 characters is

SCANDAL

In all caps and without spaces the answer would be SCANDAL
Behind, nautically The answer is AFTAST
"American Dad!" network The answer is FOX
Theme of la festa di San Valentino A lovely Italianthemed crossword clue

The answer is AMOREE
Spanish equivalent of "Basta!" The answer is ALDILA
Word with know or show The answer is TELL
Virtuoso The answer is MAE
This is only a test The answer is SIRE
Sauce that's 80% vowels A clever clue

After some thought I think I have the answer ALOEI

This sauce is a type of fruit sauce made from aloes and its indeed 80 vowels
Worshiper at the ancient Qorikancha ("Golden Temple") The answer is INCA
Core group The answer is NUB
Ending for some government 37-Across A short and sweet one

The answer is USA
Poker giveaway The answer is ANTE
"Beauty is truth, truth beauty" poet The 

Doesn't just assume, say The answer is CONSIDER
Attired The answer is DRESSED
Made level The answer to the crossword clue Made level with a length of 9 characters is

FLATTENED
Also The answer is TOO
Subj. of arms talks WAR
___ funk I think I can help you out

The answer to the crossword clue  funk with a length of 3 characters is likely THE
Hanks's "Sleepless in Seattle" co-star The answer is RYAN
Sound from a fan The answer is WHOA
Portable structure that's pitched The answer is TENT
Exist BE
Skin layer The answer is EPIDERM
Uninspiring The answer to the crossword clue Uninspiring with a length of 4 characters is

DULL
Overseer of Hamlet's duel with Laertes The answer is OSRIC
Council site of 1545 The answer is EDINB
Absorbs, as body moisture The answer is ABSORBS
"Laughing" scavenger The answer is HYENA
New Haven Ivy Leaguer The answer is YALEST
___ au lait The answer is CAFE
Begged The answer is PLEA
What the tangent of 45° is equal to A mathrelated crossword clue

The answer is SQ

Iranian money The answer is RIAL
Normandy battle site in W.W. II The answer is OMAHA
Backup singer for Gladys Knight The answer is MLD
Montezuma's people The answer is AZTECS
Only major Texas city on Mountain Time ELPASO
Performer in a kimono GEISHA
Did as directed The answer is FOLLOWED
Sweet cake topper The answer is ICING
Malted, e.g. The answer is BREWED
Spice whose name consists of two consecutive pronouns A clever clue

I think I have the answer ITISEE

The spice is ITISEE also known as Itsi or Itsi Bitsi a type of pepper The name consists of two consecutive pronouns it and is
Bark like a lap dog YIP
Fuel from a bog The answer is PEAT
North-of-the-border sports org. The answer is NHL
Fury ANG
Young fellow The answer is LAD
Good name for a museum curator? A short and sweet one

I think I have the answer ART

The wordplay is that Art is a good name for a museum curator and its a 3character word
"___ what?" I think I can help you out

The answer to the crossword clue  what with a le

...

___ and every The answer is ALLTHE
Person with a microphone The answer is SPEAKER
Finest-quality, informally The answer is AONE
Word after pen or pet The answer is FRIEND
Company behind Battlezone and Asteroids The answer is ATARI
"Wheel of Fortune" play The answer is SPIN
Late "Jeopardy!" host Alex The answer is TREBEK
V.I.P.s The answer is HIGHSOCIETY
Quibbles The answer to the crossword clue Quibbles with a length of 5 characters is

NITPICK
Stack of papers The answer is SHEETS
Stand-up comic Margaret The answer is CHO
"___ whiz!" The answer is WOW
Approx. when to get to the airport for a pickup I think I can help you with that

The answer to the crossword clue Approx when to get to the airport for a pickup with a length of 3 characters is likely TWO

So the answer in all caps and with no spaces would be TWOT
Unknown, on a sched. The answer is TBA
German "mister" The answer is HERR
"Gnarly, dude!" The answer is TUB
Bold response to a threat I think I can help you with that

The answ

1981 Stephen King thriller The answer is CARR
Certain bolt holder The answer is ANCHO
Being fixed, as a car at a garage The answer is REPAIRING
Vegan milk source ALF
Still being debugged The answer is ALPHA
Turn against The answer is REBELS
Event organizer's count The answer is ATTENDEES
___ to come The answer is ITT
Cocktail with rum, curacao and fruit juice The answer is PIRANHA
Like monarch butterflies The answer is MIGRANT
Debut album for Etta James The answer is ATLAST
Apollo's half brother The answer is ARTEMIS
Fool The answer is IDIOT
Representatives' term lengths The answer is CONGRESS
A thing in poker? I think I can help you with that

The answer to the crossword clue A thing in poker with wordplay is likely ACE
Unfortunate events, old-style I think I can help you out

The answer to the crossword clue Unfortunate events oldstyle with a length of 4 characters is CALAM

Here it is in all caps and without spaces CALAM
C.I.A. whistle-blower Philip The answer is JENK
Encountered by

___ code I think I can crack the code

The answer is MORSE
Setting for the start of "The Sound of Music" AUSTRIA
Locale traversed by Lewis and Clark: Abbr. The answer is MOO
Studio upgrade I think I can help you with that

The answer to the crossword clue Studio upgrade with a length of 10 characters is likely DIGITALRECORDING
Standouts on the pitcher's mound The answer is Aces
___ Bay (place mentioned in "(Sittin' On) The Dock of the Bay") The answer is SANFORD
Provides The answer to the crossword clue Provides with a length of 7 characters is SUPPLIES
Season ticket holder, presumably The answer is FAN
Dance traditionally performed to tell a story The answer is BALLET
Reefer The answer is SHIPPER
Wet-weather footwear The answer is WELLIES
Like someone associated with a blue, pink and white flag, for short The answer is LIBERA
Hard to pick up The answer is HEAVY
Command after a crash The answer is LANDON
"Sure, that time works for me" The answer is AOKWITH
Not handle ice well, say The 

Advantages The answer is PERKS
Not going anywhere The answer is STAY


TypeError: object of type 'float' has no len()

In [152]:
answers

['The answer is PET',
 'The answer to the crossword clue Mischiefmakers with a length of 7 characters is TRICKERS',
 'I think I can help you with that\n\nThe answer to the crossword clue It might click for a writer with a length of 3 characters is PEN',
 'SEP',
 'The answer is ECO',
 'The answer is CALTHOMPSON',
 'The answer is LARGE',
 'The answer is APE',
 'The answer to the crossword clue Haphazard with a length of 9 characters is\n\nADHOCERLY',
 'The answer is COOK',
 'I think I can help you with that\n\nThe answer to the crossword clue Inits for a theatrical hit with a length of 3 characters is likely ACT',
 'The answer is EXPELLED',
 'The answer is DONEIT',
 'The answer is BONDS',
 'A short and sweet one\n\nI think I have the answer BOOM\n\nThe wordplay is that boom can refer to a sudden increase cause of boom and also the sound of a boom as in a financial bust',
 'The answer is REASSEMBLE',
 'The answer is READY',
 'A clever crossword clue\n\nI think I have the answer MET\n\nThe

In [149]:
for i in range(len(cleaned)):
    print(cleaned[i], answers[i].translate(str.maketrans('', '', string.punctuation)))

In [153]:
with open('stripped_answers.txt', 'w') as f:
    for line in answers:
        f.write(f"{line}\n")

In [154]:
cleaned = []
for i, prompt in enumerate(df['Clue'][:250]):
    clue = df['Clue'][i]
    length = len(df['Word'][i])
    stripped = answers[i].translate(str.maketrans('', '', string.punctuation))
    cleaned.append(find_answer(stripped, length))

In [155]:
cleaned

['PET',
 'TRICKERS',
 'PEN',
 'SEP',
 'ECO',
 'CALTHOMPSON',
 'LARGE',
 'APE',
 'ADHOCERLY',
 'COOK',
 'ACT',
 'EXPELLED',
 'DONEIT',
 'BONDS',
 'BOOM',
 'REASSEMBLE',
 'READY',
 'MET',
 'ASAMIE',
 'PLATE',
 'BROO',
 'URGENTLY',
 'AUDITIONS',
 'BARTS',
 'LOOK',
 'FURLONGS',
 'LOL',
 'ALERTED',
 'HASSLE',
 'DECREED',
 'HALLS',
 'BEACHBOYSGROUP',
 'ALMOSTEXACTLY',
 'SEEM',
 'MICROPH',
 'STASIS',
 'NSA',
 'FULLTIME',
 'KING',
 'THON',
 'BASED',
 'HONORABLEDISCHARGE',
 'AREYOUT',
 'TRUE',
 'MICHAEL',
 'POPSIES',
 'UTE',
 'JOLT',
 'STONEST',
 'HEAR',
 'LEVY',
 'ERODE',
 'PEKOE',
 'NUM',
 'JIBE',
 'WHATEVERTHECASEMAYBE',
 'ACE',
 'AIMS',
 'INTERSTATE',
 'FSU',
 'YUP',
 'PIE',
 'POSSESS',
 'MAR',
 'FACETOFACE',
 'CARELES',
 'XEROX',
 'JOLLYRANCHER',
 'ALB',
 'FLAT',
 'SULIA',
 'COTTONWEAVES',
 'IES',
 'HOBBIT',
 'EPSILONGREEK',
 'BANANABOY',
 'INSECT',
 'TUC',
 'HOME',
 'MIAMI',
 'PEPE',
 'DEEPEST',
 'TALUS',
 'NEATESTLY',
 'SISTER',
 'YIN',
 'RAMONES',
 'EVIL',
 'TOP',
 'NEURON',
 'SPEND',
 

In [159]:
count = 0
for i, word in enumerate(cleaned):
    if word == df['Word'][i]:
        count += 1
print(f"Llama prediction accuracy: {count / len(cleaned)}")

Llama prediction accuracy: 0.204


In [161]:
count = 0
for i, word in enumerate(cleaned):
    if len(word) == len(df['Word'][i]):
        count += 1
print(f"Correct length predictions: {count / len(cleaned)}")

Correct length predictions: 0.556
